# Storage and transactions

In [1]:
%reload_ext pycozo.ipyext_direct

In [2]:
%cozo_auth *******

## Stored relations

An obvious shortcoming of our previous acrobatics is that we have to carry around our love triangles network and enter it anew for every query, which leads to rapid deterioration of the `CTRL`, `C` and `V` keys. So let's fix that:

In [3]:
?[a, b] <- [['alice', 'eve'],
            ['bob', 'alice'],
            ['eve', 'alice'],
            ['eve', 'bob'],
            ['eve', 'charlie'],
            ['charlie', 'eve'],
            ['david', 'george'],
            ['george', 'george']]
        
:create triangles

,status
0,OK


We have the _query directive_ `:relation create` together with a normal query. The results will then be stored on your disk with the name `triangles` instead of returned to you.

You will receive an error if you try to run this script twice. In which case don't worry and continue.

Stored relations are safe from restarts and power failures. Let's query against it:

In [4]:
?[a, b] := :triangles[a, b]

,a,b
0,alice,eve
1,bob,alice
2,charlie,eve
3,david,george
4,eve,alice
5,eve,bob
6,eve,charlie
7,george,george


The colon `:` in front of the name tells the database that we want a _stored_ relation instead of a relation defined within the query itself.

Now, Fred finally comes to the party and Fred loves Alice and Eve. We add these facts in the following way:

In [5]:
?[a, b] <- [['fred', 'alice'],
            ['fred', 'eve']]

:put triangles

,status
0,OK


In [6]:
?[a, b] := :triangles[a, b]

,a,b
0,alice,eve
1,bob,alice
2,charlie,eve
3,david,george
4,eve,alice
5,eve,bob
6,eve,charlie
7,fred,alice
8,fred,eve
9,george,george


Notice that we used `:relation put` instead of `:relation create`. In fact, you can use `:relation put` before any call to `:relation create`. The `create` op just ensures that the insertion is into a new stored relation.

Now Eve no longer loves Alice and Charlie! Let's reflect this fact by using `retract`

In [7]:
?[a, b] <- [['eve', 'charlie'],
            ['eve', 'alice']]

:rm triangles

,status
0,OK


In [8]:
?[a, b] := :triangles[a, b]

,a,b
0,alice,eve
1,bob,alice
2,charlie,eve
3,david,george
4,eve,bob
5,fred,alice
6,fred,eve
7,george,george


It is OK to retract non-existent facts, in which case the operation does nothing.

You can also reset the whole relation with `rederive`:

In [9]:
?[a, b] <- [['eve', 'charlie'],
            ['eve', 'alice']]

:replace triangles

,status
0,OK


In [10]:
?[a, b] := :triangles[a, b]

,a,b
0,eve,alice
1,eve,charlie


Only the `rederive`ed tuples remain.

You can see what stored relations you currently have in your database by running the following _system directive_:

In [11]:
::relations

,name,arity,n_keys,n_non_keys,n_put_triggers,n_rm_triggers,n_replace_triggers
0,airport,11,1,10,0,0,0
1,triangles,2,2,0,0,0,0


Relations can be renamed:

In [12]:
::relation rename triangles -> love_triangles

,status
0,OK


In [13]:
::relations

,name,arity,n_keys,n_non_keys,n_put_triggers,n_rm_triggers,n_replace_triggers
0,airport,11,1,10,0,0,0
1,love_triangles,2,2,0,0,0,0


In [14]:
?[a, b] := :love_triangles[a, b]

,a,b
0,eve,alice
1,eve,charlie


Now this triangles business is becoming tiring. Let's get rid of it:

In [15]:
::relation remove love_triangles

,status
0,OK


Since we do not have any queries to run when nuking relations, we use a system directive instead of a query directive. Now you can no longer query the triangles:

In [16]:
?[a, b] := :love_triangles[a, b]

query::relation_not_found

  × Cannot find requested stored relation 'love_triangles'


This completes all the operations on stored relations: `create`, `put`, `retract`, `rederive`. The syntax for `remove` is different from the rest for technical reasons.

All these operations are _atomic_, meaning that for all the tuples they affect, either all are affected at the same time, or the operation completely fails. There is no in-between, corrupted state.

Stored relations are simple, fast, and very raw. They can be used in exactly the same way as rules defined inline with the query. The way you use them is also not very different than in a traditional SQL database.

Stored relations are suitable for data that has a well-defined structure at the onset, and which is loaded and updated in bulk. For example, you may have obtained from domain experts an [ontology](https://www.wikiwand.com/en/Ontology_\(information_science\)) in the form of a network of metadata. The ontology comes in nice tables with clear, detailed documentation. You store this ontology as a group of stored relations, and use them to extract insights from your business data. The ontology is updated periodically, and when an update comes you just use the `rederive` operation to replace the old version. Very simple and efficient. But if you need more guarantees for your data, or your data shapes change rapidly, use the triple store instead: see a later tutorial for how to use it.